In [1]:
import lotr

In [4]:
conf = lotr.read_conf()
lotr.clear_project_folders()
lotr.download_sheet(conf)
sets = lotr.get_sets(conf)

Reading project configuration
Clearing the project folders
Getting all sets to work on


In [5]:
for set_data in sets:
    set_id, _, set_row = set_data
    print('Processing set ID {}:'.format(set_id))
    lotr.backup_previous_xml(conf, set_id)
    lotr.generate_xml(conf, set_row)
    lotr.update_xml(conf, set_id)
    lotr.calculate_hashes(set_id)
    lotr.copy_raw_images(conf, set_id)
    lotr.copy_xml(set_id)

Processing set ID 8a3273ca-1ccd-4e07-913b-766fcc49fe6f:
  Backing up a previous Strange Eons xml file
  Generating xml files for StrangeEons and OCTGN
  Updating the Strange Eons xml file with additional data
  Updating the Strange Eons xml file with hashes and skip flag
  Copying raw image files into the project folder
  Copying the Strange Eons xml file for the set into the project
Processing set ID 17b90ca1-7e3e-4891-99da-954a7833e88e:
  Backing up a previous Strange Eons xml file
  Generating xml files for StrangeEons and OCTGN
  Updating the Strange Eons xml file with additional data
  Updating the Strange Eons xml file with hashes and skip flag
  Copying raw image files into the project folder
  Copying the Strange Eons xml file for the set into the project


In [6]:
lotr.create_project()

Creating a Strange Eons project archive


### Manual Strange Eons Part:

1. Open `setGenerator.seproject` generated above.
2. Run `Script/makeCards` script by double clicking it.  This will take the templates, fill them with values from `xml` files and export results in all the required formats.
3. Once completed, close Strange Eons.

In [ ]:
conf = lotr.read_conf()
sets = lotr.get_sets(conf)

In [ ]:
for set_data in sets:
    set_id, set_name, _ = set_data
    print('Processing set ID {}:'.format(set_id))
    skip_ids = lotr.get_skip_cards(set_id)

    if 'db_octgn' in conf['outputs']:
        lotr.generate_jpg300_nobleed(set_id, skip_ids)
    if 'pdf' in conf['outputs']:
        lotr.generate_png300_pdf(conf, set_id, skip_ids)
    if 'makeplayingcards' in conf['outputs']:
        lotr.generate_png800_bleedmpc(conf, set_id, skip_ids)

    if 'db' in conf['outputs']:
        lotr.generate_db(set_id, set_name)
    if 'octgn' in conf['outputs']:
        lotr.generate_octgn(set_id)
    if 'pdf' in conf['outputs']:
        lotr.generate_pdf(set_id, set_name)
    if 'makeplayingcards_zip' in conf['outputs']:
        lotr.generate_mpc_zip(set_id, set_name)
    if 'makeplayingcards_7z' in conf['outputs']:
        lotr.generate_mpc_7z(set_id, set_name)

Now there should be the following outputs:

1. `Output/DB/<set name>/`: 300 dpi JPG images for general purposes.
2. `Output/MakePlayingCards/<set name>`: `zip` and `7z` archives of 800 dpi PNG images
    to be printed on MakePlayingCards.com.
3. `Output/OCTGN/<octgnid>/:` `set.xml` and `o8c` image pack (300 dpi JPG) for OCTGN.
    Add the latter using the "Add Image Packs" button from within OCTGN.
4. `Output/PDF/<set name>/`: PDF files in `A4` and `letter` format for home printing.